In [ ]:
import argparse
import pathlib
import json
from collections import defaultdict

import pymongo
from pymongo import MongoClient

import pandas as pd

from tqdm import tqdm

from util import serialize_parquet
from util import create_collection

In [ ]:
data_path = pathlib.Path("~/mnt/seagate/jkleinhenz/projects/aimm/wanli/core/").expanduser()
print(data_path)

In [ ]:
mongo_uri = None
db = "aimm"
collection = "wanli"

In [ ]:
client = MongoClient(mongo_uri)
db = client[db]

with open("schema.json") as f:
   schema = json.load(f)
c = create_collection(db, collection, schema, overwrite=True)

In [ ]:
def spike():
  files = (data_path / "O_K").glob("*.txt")
  for f in files:
    df = pd.read_csv(f, header=None, delimiter="\t", names=["energy", "mu"])
    
    element = {"symbol" : "O", "edge" : "K"}
    common = {
      "element" : element,
      "spec" : "wanli",
      "uid" : f.stem}
    
    metadata = {"common" : common}
    
    data = {
        "media_type": "application/x-parquet",
        "structure_family": "dataframe",
        "blob": serialize_parquet(df).tobytes(),
    }
    
    doc = {"data": data, "metadata": metadata}
    c.insert_one(doc)      
  
spike()

In [ ]:
def load_TM_L():
  files = (data_path / "TM_L")
  
  for d in files.iterdir():
    if d.is_file():
      continue
    symbol = d.stem
    edge = "L"
    element = {"symbol" : symbol, "edge" : edge}
    
    for f in d.glob("*.txt"):
      if f.stem.startswith("IgorPlot"):
        continue
        
      print(f.stem)
      
      with open(f, "r") as ff:
        l = ff.readline()
        n = len(l.split())
        
        if n == 1:
          skiprows = 1
          title = l
        elif n == 2:
          skiprows = 0
          title = None
        else:
          assert False
          
      df = pd.read_csv(f, delimiter="\t", names=["energy", "mu"], skiprows=skiprows)
      
      common = {
        "element" : element,
        "spec" : "wanli",
        "uid" : f.stem}
      
      metadata = {"common" : common}
      
      data = {
          "media_type": "application/x-parquet",
          "structure_family": "dataframe",
          "blob": serialize_parquet(df).tobytes(),
      }
      
      doc = {"data": data, "metadata": metadata}
      c.insert_one(doc)      

load_TM_L()